<a href="https://colab.research.google.com/github/davidsjohnson/evalxai_studies/blob/main/two4two_sickones_explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pathlib
import os
from pathlib import Path
import random
import datetime

import numpy as np
from PIL import Image

from skimage.transform import resize

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Params

In [3]:
load_checkpoints = True

modeltype = 'mobilenet'

biased_ds = 'sick_ones_bendcolorbias'
nobias_ds = 'sick_ones_bendbias'

epochs = 10
learning_rate = 0.0001

# Setup and Load Datasets

In [4]:
def load_dataframe(data_dir, dataset):
  data_dir = data_dir / dataset
  df = pd.read_json(data_dir / 'parameters.jsonl', lines=True)
  df['filename'] = df['id'] + '.png'
  df['ill'] = df['ill'].astype(int).astype(str)
  return df

def load_data_gens(trainval_datadir, test_datadir):
  train_df = load_dataframe(trainval_datadir, 'train')
  valid_df = load_dataframe(trainval_datadir, 'validation')
  test_df = load_dataframe(test_datadir, 'test')

  datagen = ImageDataGenerator(rescale=1./255)
  train_generator = datagen.flow_from_dataframe(dataframe=train_df, directory=trainval_datadir / 'train', target_size=(128, 128),
                                                x_col='filename', y_col='ill', batch_size=64, shuffle=True)
  valid_generator = datagen.flow_from_dataframe(dataframe=valid_df, directory=trainval_datadir / 'validation', target_size=(128, 128),
                                                x_col='filename', y_col='ill', batch_size=64, shuffle=False)
  test_generator = datagen.flow_from_dataframe(dataframe=test_df, directory=test_datadir / 'test',  target_size=(128, 128),
                                               x_col='filename', y_col='ill',
                                               batch_size=64, shuffle=False)

  return train_generator, valid_generator, test_generator

In [5]:
gdrive = Path('/content/drive/MyDrive')
relative_model_path = "two4two_sickones_models"
base_path = gdrive / relative_model_path

output = gdrive / 'hcxai' / 'blocky_diagnosis'

base_path

PosixPath('/content/drive/MyDrive/two4two_sickones_models')

In [6]:
data_dir_sickones = keras.utils.get_file(
    origin = "https://uni-bielefeld.sciebo.de/s/Ve8WuZv3teRtVhG/download",
    fname = 'two4two_datasets.tar.gz',
    extract = True,
    archive_format = 'tar'
)
data_dir_sickones = Path(data_dir_sickones)

3837766635/3837766635 [==============================] - 245s 0us/step


## Load Biased and Unbiased datasets

In [7]:
nobias_dir = data_dir_sickones.with_suffix('').with_suffix('') / nobias_ds
biased_dir = data_dir_sickones.with_suffix('').with_suffix('') / biased_ds

nobias_dir, biased_dir

(PosixPath('/root/.keras/datasets/two4two_datasets/sick_ones_bendbias'),
 PosixPath('/root/.keras/datasets/two4two_datasets/sick_ones_bendcolorbias'))

In [8]:
nobias_model_path = base_path / nobias_ds / f'{modeltype}'
biased_model_path = base_path / biased_ds / f'{modeltype}'

nobias_model_path, biased_model_path

(PosixPath('/content/drive/MyDrive/two4two_sickones_models/sick_ones_bendbias/mobilenet'),
 PosixPath('/content/drive/MyDrive/two4two_sickones_models/sick_ones_bendcolorbias/mobilenet'))

In [9]:
nobias_model_exists = os.path.exists(nobias_model_path)
biased_model_exists = os.path.exists(biased_model_path)

nobias_model_exists, biased_model_exists

(True, True)

In [10]:
# Load Dataframes
nobias_train_df,  nobias_valid_df, nobias_test_df = (load_dataframe(nobias_dir, 'train'),
                                                     load_dataframe(nobias_dir, 'validation'),
                                                     load_dataframe(nobias_dir, 'test'))
nobias_train_df['sphere_diff'] = np.abs(nobias_train_df['spherical'] - nobias_train_df['ill_spherical'])
nobias_valid_df['sphere_diff'] = np.abs(nobias_valid_df['spherical'] - nobias_valid_df['ill_spherical'])
nobias_test_df['sphere_diff'] = np.abs(nobias_test_df['spherical'] - nobias_test_df['ill_spherical'])

biased_train_df,  biased_valid_df, biased_test_df = (load_dataframe(biased_dir, 'train'),
                                                     load_dataframe(biased_dir, 'validation'),
                                                     load_dataframe(biased_dir, 'test'))
biased_train_df['sphere_diff'] = np.abs(biased_train_df['spherical'] - biased_train_df['ill_spherical'])
biased_valid_df['sphere_diff'] = np.abs(biased_valid_df['spherical'] - biased_valid_df['ill_spherical'])
biased_test_df['sphere_diff'] = np.abs(biased_test_df['spherical'] - biased_test_df['ill_spherical'])

In [11]:
# Load Data Gens
nobias_train_gen, nobias_valid_gen, nobias_test_gen = load_data_gens(nobias_dir, nobias_dir)
biased_train_gen, biased_valid_gen, biased_test_gen = load_data_gens(biased_dir, biased_dir)

Found 80000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.
Found 3000 validated image filenames belonging to 2 classes.
Found 80000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.
Found 3000 validated image filenames belonging to 2 classes.


# Model Loading and Evaluation

In [12]:
def get_model(load_checkpoint: bool, model_filepath: Path, learning_rate: float):

  if load_checkpoint:
    model = keras.models.load_model(model_filepath)
    print(f'loading existing checkpoint for mobilenet - {model_filepath}')
  else:
    print('Model does not exist or checkpoint not set to be loaded. Loading new mobilenet.')
    base_model = keras.applications.MobileNetV2(
        input_shape=(128, 128, 3),
        alpha=1.0,
        include_top=False,
        weights=None,
        input_tensor=None,
        pooling='avg'
    )

    model = keras.Sequential([
        base_model,
        layers.Dense(2, activation="softmax"),
    ])

    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt, metrics=["accuracy"])

  return model

def get_simple_model(load_checkpoint: bool, model_filepath: Path, learning_rate: float):


  if load_checkpoint:
    model = keras.models.load_model(model_filepath)
    print(f'loading existing checkpoint for simple net- {model_filepath}')
  else:
    print('Model does not exist or checkpoint not set to be loaded. Loading new simple net.')
    model = keras.models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(2, activation="softmax"),
    ])

    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt, metrics=["accuracy"])

  return model

In [13]:
def eval(model, test_generator):

  print(f'Evaluating model on test data')
  return model.evaluate(test_generator)[1]

## Load Model

### No Bias Model

In [14]:
if modeltype == 'mobilenet':
  nobias_model = get_model(load_checkpoint=True,
                           model_filepath=nobias_model_path,
                           learning_rate=learning_rate)
elif modeltype == 'simple':
  nobias_model = get_simple_model(load_checkpoint=True,
                                  model_filepath=nobias_model_path,
                                  learning_rate=learning_rate)
else:
  print('Model type does not exist')
  nobias_model = None

loading existing checkpoint for mobilenet - /content/drive/MyDrive/two4two_sickones_models/sick_ones_bendbias/mobilenet


In [15]:
nobias_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Func  (None, 1280)              2257984   
 tional)                                                         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2226434 (8.49 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________


### Biased Model

In [16]:
if modeltype == 'mobilenet':
  biased_model = get_model(load_checkpoint=True,
                           model_filepath=biased_model_path,
                           learning_rate=learning_rate)
elif modeltype == 'simple':
  biased_model = get_simple_model(load_checkpoint=True,
                                  model_filepath=biased_model_path,
                                  learning_rate=learning_rate)
else:
  print('Model type does not exist')
  biased_model = None

loading existing checkpoint for mobilenet - /content/drive/MyDrive/two4two_sickones_models/sick_ones_bendcolorbias/mobilenet


In [17]:
biased_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Func  (None, 1280)              2257984   
 tional)                                                         
                                                                 
 dense_1 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2226434 (8.49 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________


## Eval  Model

### No Bias Model

In [18]:
# results on nobias test
acc = eval(nobias_model, nobias_test_gen)
print(f'ACC: {acc}')
probs = nobias_model.predict(nobias_test_gen)
preds = np.argmax(probs, axis=-1)

nobias_test_df['nobias_pred'] = preds
nobias_test_df['nobias_pred'] = nobias_test_df['nobias_pred'].astype(str)
(nobias_test_df['nobias_pred'] == nobias_test_df['ill']).astype(int).mean()

Evaluating model on test data
47/47 [==============================] - 13s 249ms/step - loss: 0.4659 - accuracy: 0.8113
ACC: 0.8113333582878113
47/47 [==============================] - 13s 248ms/step


0.8113333333333334

In [19]:
# results on biased test
acc = eval(nobias_model, biased_test_gen)
print(f'ACC: {acc}')
probs = nobias_model.predict(biased_test_gen)
preds = np.argmax(probs, axis=-1)

biased_test_df['nobias_pred'] = preds
biased_test_df['nobias_pred'] = biased_test_df['nobias_pred'].astype(str)
(biased_test_df['nobias_pred'] == biased_test_df['ill']).astype(int).mean()

Evaluating model on test data
47/47 [==============================] - 12s 249ms/step - loss: 0.4965 - accuracy: 0.7903
ACC: 0.7903333306312561
47/47 [==============================] - 12s 248ms/step


0.7903333333333333

### Biased Model

In [20]:
# results on nobias test
acc = eval(biased_model, nobias_test_gen)
print(f'ACC: {acc}')
probs = biased_model.predict(nobias_test_gen)
preds = np.argmax(probs, axis=-1)

nobias_test_df['biased_pred'] = preds
nobias_test_df['biased_pred'] = nobias_test_df['biased_pred'].astype(str)
(nobias_test_df['biased_pred'] == nobias_test_df['ill']).astype(int).mean()

Evaluating model on test data
47/47 [==============================] - 13s 257ms/step - loss: 0.5248 - accuracy: 0.7883
ACC: 0.7883333563804626
47/47 [==============================] - 13s 245ms/step


0.7883333333333333

In [21]:
# results on biased test
acc = eval(biased_model, biased_test_gen)
print(f'ACC: {acc}')
probs = biased_model.predict(biased_test_gen)
preds = np.argmax(probs, axis=-1)

biased_test_df['biased_pred'] = preds
biased_test_df['biased_pred'] = biased_test_df['biased_pred'].astype(str)
(biased_test_df['biased_pred'] == biased_test_df['ill']).astype(int).mean()

Evaluating model on test data
47/47 [==============================] - 13s 266ms/step - loss: 0.4196 - accuracy: 0.8243
ACC: 0.8243333101272583
47/47 [==============================] - 13s 266ms/step


0.8243333333333334

# Explanations

## Generate Explanation Dataset

Select 20 images to represent challenging decision making tasks but where there is a clear final decision.

(for now from nobias dataset with biased model predictions)

- 16 - Correct predictions
- 4 - Incorrect predictions

represents accuracy of model (around $80\%$)

(actually I think this does not matter.  a small sample of 20 samples will be the same as teh original model, especially in hard cases.

Will instead pick 6 total from each category

- Correct Predictions (3 each Correct and 3 Incorrect)
  - medhigh sphere diff and lowmed bend (**overlap** - ill and notill)
  - lowmed sphere diff but medhigh bend (**overlap** - ill and notill)
  - medhigh sphere diff and medhigh bend (**easy** - ill)
  - lowmed sphere diff and lowmed bend (**easy** - ill)
  - lowmed sphere diff but (1,3) changed pieces (**slightly difficult** - does XAI help show ambiguous pieces)


In [22]:
def display_images(df, n_rows, n_cols, img_dir, title, random_state=0):
  """ Function to display images in a grid randomly selected from a dataframe of images.

  Args:
    df (pd.DataFrame): dataframe of images
    n_rows (int): number of rows in the grid
    n_cols (int): number of columns in the grid
    title (str): title of the plot
    random_state (int): random state for reproducibility
  """

  if n_rows == 0 and n_cols == 0:
    print(f'Not data to display for Figure - {title}')
    return

  if n_rows * n_cols < len(df):
    df = df.sample(n_rows * n_cols, random_state=random_state)

  figsize = (n_cols * 2, n_rows*2.5)
  print(figsize)

  test_images = np.array([Image.open(p).convert('RGB') for p in img_dir / df['filename']]) * 1. / 255
  fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
  axes = np.array(axes)
  for i, (ax, idx) in enumerate(zip(axes.flat, df.index)):
    ax.imshow(test_images[i])
    ax.set_title(f'Img Idx {idx}')
    ax.axis('off')

  fig.suptitle(title)
  fig.tight_layout()

### Correct Samples Selection


In [51]:
# setup conditions
def get_xai_conds(df, ill=False):

  """
    - medhigh sphere diff and lowmed bend (**overlap** - ill and notill)
    - lowmed sphere diff but medhigh bend (**overlap** - ill and notill)
    - medhigh sphere diff and medhigh bend (**easy** - ill)
    - lowmed sphere diff and lowmed bend (**easy** - notill)
    - lowmed sphere diff but (1,3) changed pieces (**slightly difficult** - does XAI help show ambiguous pieces)
  """

  medhigh_bend = (abs(df['bending']).quantile(0.50), abs(df['bending']).quantile(0.70))
  medhigh_diff = (abs(df['sphere_diff']).quantile(0.50), abs(df['sphere_diff']).quantile(0.70))

  lowmed_bend = (abs(df['bending']).quantile(0.30), abs(df['bending']).quantile(0.40))
  lowmed_diff = (abs(df['sphere_diff']).quantile(0.30), abs(df['sphere_diff']).quantile(0.40))

  low_diff = (0, abs(df['sphere_diff']).quantile(0.30))

  cond_medhigh_bend = (abs(df['bending']) > medhigh_bend[0]) & (abs(df['bending']) < medhigh_bend[1])
  cond_medhigh_diff = (abs(df['sphere_diff']) > medhigh_diff[0]) & (abs(df['sphere_diff']) < medhigh_diff[1])

  cond_lowmed_bend = (abs(df['bending']) > lowmed_bend[0]) & (abs(df['bending']) < lowmed_bend[1])
  cond_lowmed_diff = (abs(df['sphere_diff']) > lowmed_diff[0]) & (abs(df['sphere_diff']) < lowmed_diff[1])

  cond_low_diff = (abs(df['sphere_diff']) > low_diff[0]) & (abs(df['sphere_diff']) < low_diff[1])

  num_diff_is2 = (df['num_diff'] == 2)

  cond1 = (cond_medhigh_diff & cond_lowmed_bend & num_diff_is2)
  cond2 = (cond_lowmed_diff & cond_medhigh_bend & num_diff_is2)
  cond3 = (cond_medhigh_diff & cond_medhigh_bend & num_diff_is2)
  cond4 = (cond_lowmed_diff & cond_lowmed_bend & num_diff_is2)
  cond5 = (cond_lowmed_diff & cond_medhigh_bend)

  return cond1, cond2, cond3, cond4, cond5

In [52]:
ill_rows = nobias_test_df['ill'] == '1'
notill_rows = nobias_test_df['ill'] == '0'

correct_rows = nobias_test_df['biased_pred'] == nobias_test_df['ill']
incorrect_rows = nobias_test_df['biased_pred'] != nobias_test_df['ill']

num_diff_is1 = (nobias_test_df['num_diff'] == 1)
num_diff_is3 = (nobias_test_df['num_diff'] == 3)

cond1, cond2, cond3, cond4, cond5 = get_xai_conds(nobias_test_df)

In [80]:
test_df = nobias_test_df[cond1 | cond2 | cond3 | cond4 | (cond5 & num_diff_is1) | (cond5 & num_diff_is3)]
test_df = test_df.sample(30, random_state=5)

correct_rows = test_df['nobias_pred'] == test_df['ill']
incorrect_rows = test_df['nobias_pred'] != test_df['ill']

ill_rows = test_df['ill'] == '1'
notill_rows = test_df['ill'] == '0'

num_diff_is1 = (test_df['num_diff'] == 1)
num_diff_is3 = (test_df['num_diff'] == 3)

cond1, cond2, cond3, cond4, cond5 = get_xai_conds(test_df)

In [81]:
((test_df['biased_pred'] == test_df['ill']).astype(int).mean(),
  (test_df['nobias_pred'] == test_df['ill']).astype(int).mean())

(0.6333333333333333, 0.6666666666666666)

In [82]:
def print_cond_stats(cond, title):
  print(title)
  print('Ill',
        abs(test_df[cond & ill_rows]['sphere_diff']).mean(),
        abs(test_df[cond & ill_rows]['bending']).mean(),
        abs(test_df[cond & ill_rows]['num_diff']).mean(),
        abs(test_df[cond & ill_rows]['num_diff']).count()
        )
  print('Not Ill',
        abs(test_df[cond & notill_rows]['sphere_diff']).mean(),
        abs(test_df[cond & notill_rows]['bending']).mean(),
        abs(test_df[cond & notill_rows]['num_diff']).mean(),
        abs(test_df[cond & notill_rows]['num_diff']).count()
        )
  print('\n')


In [85]:
print_cond_stats((incorrect_rows & cond1), 'Correct Condition 1 - medhigh sphere diff and medlow bend')
print_cond_stats((incorrect_rows & cond2), 'Correct Condition 2 - medhigh sphere diff and medhigh bend')
print_cond_stats((incorrect_rows & cond3), 'Correct Condition 3 - medlow sphere diff but medhigh bend')
print_cond_stats((incorrect_rows & cond4), 'Correct Condition 4 - low sphere diff but (1,3) changed pieces')
print_cond_stats((incorrect_rows & cond5 & (num_diff_is1 | num_diff_is3)), 'Correct Condition 5 - low sphere diff but (1,3) changed pieces')

Correct Condition 1 - medhigh sphere diff and medlow bend
Ill nan nan nan 0
Not Ill nan nan nan 0


Correct Condition 2 - medhigh sphere diff and medhigh bend
Ill nan nan nan 0
Not Ill nan nan nan 0


Correct Condition 3 - medlow sphere diff but medhigh bend
Ill nan nan nan 0
Not Ill 0.423738820843456 0.237998045175586 2.0 1


Correct Condition 4 - low sphere diff but (1,3) changed pieces
Ill nan nan nan 0
Not Ill nan nan nan 0


Correct Condition 5 - low sphere diff but (1,3) changed pieces
Ill nan nan nan 0
Not Ill nan nan nan 0




In [29]:
# Cond 1 - Correct Ill Examples
display_images(nobias_test_df[(correct_rows & ill_rows & cond1)], 2, 5, nobias_dir / 'test', 'Correct Ill Cond1 - medhigh sphere diff and medlow bend', random_state=0)

(10, 5.0)


<ipython-input-23-ca6a5cf77f44>:22: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  test_images = np.array([Image.open(p).convert('RGB') for p in img_dir / df['filename']]) * 1. / 255


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [ ]:
ill_cond1_corr_idxs = [891, 1167] # rand state = 0

In [ ]:
# Cond 1 - Correct Not Ill Examples
display_images(nobias_test_df[(correct_rows & notill_rows & cond1)], 2, 5, nobias_dir / 'test', 'Correct Not Ill Cond1 - medhigh sphere diff and medlow bend', random_state=0)

In [ ]:
notill_cond1_corr_idxs = [1520, 357] # rand state = 1

In [ ]:
# Cond2 - Correct Ill Examples
display_images(nobias_test_df[correct_rows & ill_rows & cond2], 2, 5, nobias_dir / 'test', 'Correcct Ill Cond2 - medhigh sphere diff and medhigh bend', random_state=1)

In [ ]:
ill_cond2_corr_idxs = [1332, 1533] # rand state = 1

In [ ]:
# Cond2 - Correct Not Ill Examples
display_images(nobias_test_df[correct_rows & notill_rows & cond2], 2, 5, nobias_dir / 'test', 'Correct Not Ill Cond2 - medhigh sphere diff and medhigh bend', random_state=1)

In [ ]:
notill_cond2_corr_idxs = [2085, 512] # rand state = 1

In [ ]:
# Cond3 - Correct Ill Examples
display_images(nobias_test_df[correct_rows & ill_rows & cond3], 2, 5, nobias_dir / 'test', 'Correct Ill Cond3 - medlow sphere diff but medhigh bend', random_state=0)

In [ ]:
ill_cond3_corr_idxs = [2765, 1856] # rand state = 0

In [ ]:
# Cond3 - Correct Not Ill Examples
display_images(nobias_test_df[correct_rows & notill_rows & cond3], 2, 5, nobias_dir / 'test', 'Correct Not Ill Cond3 - lowmed sphere diff but medhigh bend', random_state=32)

In [ ]:
notill_cond3_corr_idxs = [2172, 189] # rand state = 32

In [ ]:
# Condition 4 - Correct Ill Examples
display_images(nobias_test_df[correct_rows & ill_rows & cond4 & num_diff_is3], 2, 5, nobias_dir / 'test', 'Correct Ill Cond4 - low sphere diff, medhigh bend, and 3 changed', random_state=1)

In [ ]:
ill_cond4_corr_idxs = [2531, 1859] # rand state = 32, 1

In [ ]:
# Condition 4 - Correct Not Ill Examples
display_images(nobias_test_df[correct_rows & notill_rows & cond4 & num_diff_is1], 2, 5, nobias_dir / 'test', 'Correct Not Ill Cond4 - low sphere diff and 1 change', random_state=11)

In [ ]:
notill_cond4_corr_idxs = [2829, 2463] # rand state = 11

### Incorrect Samples Selection

In [ ]:
print_cond_stats((incorrect_rows & cond1), 'Correct Condition 1 - medhigh sphere diff and medlow bend')
print_cond_stats((incorrect_rows & cond2), 'Correct Condition 2 - medhigh sphere diff and medhigh bend')
print_cond_stats((incorrect_rows & cond3), 'Correct Condition 3 - medlow sphere diff but medhigh bend')
print_cond_stats((incorrect_rows & cond4), 'Correct Condition 4 - low sphere diff but (1,3) changed pieces')

In [ ]:
# Condition 1 - Ill
display_images(nobias_test_df[(incorrect_rows & ill_rows & cond1)], 2, 5, nobias_dir / 'test', 'Incorrect Ill Cond1 - medhigh sphere diff and medlow bend', random_state=0)

In [ ]:
ill_cond1_inc_idxs = [702] # rand state = 0

In [ ]:
# Condition 1 - Not Ill
display_images(nobias_test_df[(incorrect_rows & notill_rows & cond1)], 2, 5, nobias_dir / 'test', 'Incorrect Not Ill Cond1 - medhigh sphere diff and medlow bend', random_state=0)


In [ ]:
notill_cond1_inc_idxs = [] # no selection for now

In [ ]:
# Condition 2 - Ill
display_images(nobias_test_df[(incorrect_rows & ill_rows & cond2)], 2, 5, nobias_dir / 'test', 'Incorrect Ill Cond2 - medhigh sphere diff and medlow bend', random_state=0)

In [ ]:
ill_cond2_inc_idxs = [] # no selection

In [ ]:
# Condition 2 - Not Ill
display_images(nobias_test_df[(incorrect_rows & notill_rows & cond2)], 2, 5, nobias_dir / 'test', 'Incorrect Not Ill Cond2 - medhigh sphere diff and medlow bend', random_state=10)

In [ ]:
notill_cond2_inc_idxs = [2625] # rand state = 10

In [ ]:
# Condition 3 - Ill
display_images(nobias_test_df[(incorrect_rows & ill_rows & cond3)], 2, 5, nobias_dir / 'test', 'Incorrect Ill Cond3 - medlow sphere diff and medhigh bend', random_state=0)

In [ ]:
ill_cond3_inc_idxs = [] # no selection

In [ ]:
# Condition 3 - Not Ill
display_images(nobias_test_df[(incorrect_rows & notill_rows & cond3)], 2, 5, nobias_dir / 'test', 'Incorrect Not Ill Cond3 - medlow sphere diff and medhigh bend', random_state=5)

In [ ]:
notill_cond3_inc_idxs = [2876] # rand state = 5

In [ ]:
# Condition 4 - Ill
display_images(nobias_test_df[(incorrect_rows & ill_rows & cond4 & num_diff_is3)], 2, 5, nobias_dir / 'test', 'Incorrect Ill Cond4 - lows sphere diff and 3 changed', random_state=0)

In [ ]:
ill_cond4_inc_idxs = [] # no selection

In [ ]:
# Condition 4 - Not Ill
display_images(nobias_test_df[(incorrect_rows & notill_rows & cond4 & num_diff_is1)], 2, 5, nobias_dir / 'test', 'Incorrect Not Ill Cond4 - low sphere diff and 1 changed', random_state=1)

In [ ]:
notill_cond4_inc_idxs = [2334] # rand state = 0

### Combine Selections

In [ ]:
corr_samples = (ill_cond1_corr_idxs + notill_cond1_corr_idxs +
                ill_cond2_corr_idxs + notill_cond2_corr_idxs +
                ill_cond3_corr_idxs + notill_cond3_corr_idxs +
                ill_cond4_corr_idxs + notill_cond4_corr_idxs)
inc_samples = (ill_cond1_inc_idxs + notill_cond1_inc_idxs +
               ill_cond2_inc_idxs + notill_cond2_inc_idxs +
               ill_cond3_inc_idxs + notill_cond3_inc_idxs +
               ill_cond4_inc_idxs + notill_cond4_inc_idxs)

assert len(set(corr_samples + inc_samples)) == 20, 'selection may contain duplicates'

corr_df = nobias_test_df.loc[corr_samples]
inc_df = nobias_test_df.loc[inc_samples]

final_xai_df = pd.concat([corr_df, inc_df])
len(final_xai_df), (final_xai_df['biased_pred'] == final_xai_df['ill']).astype(int).mean()

In [ ]:
display_images(final_xai_df, 4, 5, nobias_dir / 'test', 'Final XAI Dataset')

In [ ]:
xai_images = np.array([Image.open(p).convert('RGB') for p in nobias_dir / 'test' / final_xai_df['filename']]) * 1. / 255

#### Save Inputs

In [ ]:
def show_blocky(image, id):

  figsize = [5, 5]
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=figsize)

  #plot the input image
  input_img = resize(image, (256, 256))
  ax.imshow(image)
  ax.set_title(f'Blocky ID: {id}')
  ax.axis('off')

In [ ]:
today = datetime.datetime.today()
date_str = today.strftime('%Y-%m-%d')

o = output / 'original_input' / 'biased_model' / date_str / 'xai_samples'
os.makedirs(o, exist_ok=True)
print(f'saving to {o}')

for i, ((idx, row), image) in enumerate(zip(final_xai_df.iterrows(), xai_images)):
  print(i+1)
  show_blocky(image, row["id"])
  plt.savefig(o / f'{row["id"]}_true={row["ill"]}_pred={row["biased_pred"]}_input.png')
  plt.show()

## Explanations of Biased Model

### SHAP Explanations

In [66]:
# load background images
CLASSES = ['Not Ill', 'Ill']

n_bck = 250

bck_files = nobias_test_df['filename'].sample(n_bck, random_state=5)
bck_images = np.array([Image.open(p).convert('RGB') for p in nobias_dir / 'test' / bck_files]) * 1. / 255

In [72]:
# adding input shape to model for shap - https://github.com/shap/shap/issues/1226
shap_model_input = tf.keras.layers.Input(shape=(128, 128, 3))
shap_model_output = biased_model(shap_model_input)
shap_model = tf.keras.models.Model(inputs=shap_model_input, outputs=shap_model_output)

# Partition Explaniner
# define a masker that is used to mask out partitions of the input image.
masker = shap.maskers.Image("blur(128,128)", xai_images[0].shape)
explainer = shap.Explainer(biased_model, masker, output_names=CLASSES, max_evals=1000)

# Deep Explainer
# shap.explainers._deep.deep_tf.op_handlers["Relu6"] = shap.explainers._deep.deep_tf.nonlinearity_1d(0)
# shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.linearity_1d(0)
# shap.explainers._deep.deep_tf.op_handlers["StridedSlice"] = shap.explainers._deep.deep_tf.passthrough
# shap.explainers._deep.deep_tf.op_handlers["DepthwiseConv2dNative"] = shap.explainers._deep.deep_tf.linearity_1d(0)


# explainer = shap.DeepExplainer(shap_model, bck_images)

In [73]:
xai_images.dtype

dtype('float64')

In [74]:
shap_values = explainer(xai_images)

NotImplementedError: Cannot convert a symbolic tf.Tensor (sequential_1_2/dense_1/Softmax:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.

In [ ]:
# reshape data to fit image_plot
shap_values_t = np.transpose(shap_values.values, (-1, 0, 1, 2, 3))
shap_values_t = list(shap_values_t)

NOTE: Ill ones are biased towards red in the training data



In [ ]:
img_labels = [(CLASSES[int(t)], CLASSES[int(p)]) for t, p in zip(final_xai_df['ill'], final_xai_df['biased_pred'])]
shap.image_plot(shap_values_t, xai_images, labels=np.repeat([CLASSES], len(xai_images), axis=0), true_labels=img_labels, show=False)

# shap.image_plot(shap_values, true_labels=img_labels, show=False)
# plt.savefig('model_nobias_data_nobias.png')
plt.show()

### Innvestigate Explanations

In [70]:
np.__version__

'1.25.2'

In [71]:
tf.__version__

'2.14.1'